In [ ]:
# !pip install numpy matplotlib
!conda install pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch

# Sử dụng CPU numpy

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 1000)
y = np.sin(x)
plt.plot(x,y)
plt.show()
# Weight
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # function y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3
    loss_ = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss_)
    grad_y_pred = 3.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')


# Sử dụng torch

In [ ]:
import torch
import matplotlib.pyplot as plt

# device = torch.device("cuda:0")
device = torch.device("cpu")

dtype = torch.float

x = torch.linspace(-math.pi, math.pi, 1000, device=device, dtype=dtype)
y = torch.sin(x)
plt.plot(x,y)
plt.show()
# Weight
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6

for t in range(2000):
    y_pred = a + b * x + c * x ** 2 + d * x ** 3
    loss_ = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss_)
    grad_y_pred = 3.0 * (y_pred - y)

    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

    

In [ ]:
import torch 
print(torch.cuda.is_available())

torch.cuda.current_device()

In [ ]:
import torch
import math
x = torch.linspace(-math.pi, math.pi, 1000)
y = torch.sin(x) 

p = torch.tensor([1, 2, 3])
x_pow = x.unsqueeze(-1).pow(p)
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6

for t in range (2000):
    y_pred = model(x_pow)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    model.zero_grad()
    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

print(param)
print(x_pow.shape)
# print(model)

# Build the neural network with pytorch

In [ ]:
# !python -m pip install --upgrade pip
# !pip list

In [35]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
# from torchvision import datasets, transforms
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [39]:
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten() # lớp chuyển ảnh 2d từ 28 x 28 về mảng 1 chiều có độ dài 28*28
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512), 
        )
    
    def forward(self, x):
        x = self.flatten(x)
        print(x)
        logits = self.linear_relu_stack(x)
        return logits

In [40]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4096, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
  )
)


In [41]:
X = torch.rand(1, 64, 64, device=device)
print(X.shape)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

torch.Size([1, 64, 64])
tensor([[0.6967, 0.2669, 0.3218,  ..., 0.0732, 0.0918, 0.7192]])
Predicted class: tensor([83])


In [42]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4096, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 4096]) | Values : tensor([[-0.0068,  0.0007,  0.0134,  ..., -0.0093, -0.0113, -0.0002],
        [-0.0146,  0.0122, -0.0046,  ..., -0.0062,  0.0072, -0.0043]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0045, -0.0053], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0132, -0.0094, -0.0303,  ...,  0.0201, -0.0162,  0.0142],
        [ 0.0345,  0.0239,  0.0218,  ...,  0.0210,  0.0298, -0.0051]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias